<span style="font-family: Arial; font-size: 14pt;"><b>Finding Top Similar Words as Base Words</b></span><br>
Author: Lucas Ma<br>

In [2]:
from transformers import AutoTokenizer, AutoModel
import string, heapq, numpy as np
from torch.nn.functional import cosine_similarity
from nltk.corpus import stopwords
import os, json
import torch
import nltk
from nltk.tokenize import sent_tokenize

# Make sure to download the punkt tokenizer
nltk.download('punkt')

model_name = "emanjavacas/MacBERTh"
# model_name = "fine-tuned-MacBERTh"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /Users/lucasma/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/Users/lucasma/miniforge3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [37]:
def read_sentence_document(document_path):
    encodings = ['utf-8', 'latin-1', 'cp1252']
    for encoding in encodings:
        try:
            with open(document_path, 'r', encoding=encoding) as f:
                text = f.read()
            break  # Exit the loop if no exception was raised
        except UnicodeDecodeError:
            continue
    else:
        raise UnicodeDecodeError("Failed to read the file with any of the tried encodings.")
        
    text = text.lower()

    sentences = sent_tokenize(text)

    # Further split the sentences if there are more than 450 words
    split_sentences = []
    for sentence in sentences:
        words = sentence.split()
        while len(words) > 400:
            split_sentences.append(' '.join(words[:400]))
            words = words[400:]
        split_sentences.append(' '.join(words))

    # Strip whitespace and filter out empty sentences
    final_sentences = [s.strip() for s in split_sentences if s.strip()]
    
    return final_sentences


# Function to get word embeddings
def get_word_embedding(chunks, tokenizer, model):
    word_times = {}
    word_embeddings = {}
    for chunk in chunks:

        inputs = tokenizer(chunk, return_tensors='pt', padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = outputs.last_hidden_state
        for i, word in enumerate(tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])):
            if word not in word_times:
                word_times[word] = 1
            else:
                word_times[word]+=1
            if word not in word_embeddings:
                word_embeddings[word] = embeddings[0, i, :].numpy()
            else:
                word_embeddings[word] = (word_embeddings[word] * (word_times[word]-1) + embeddings[0, i, :].numpy()) / word_times[word]
    return word_embeddings

def get_single_embedding(word, tokenizer, model):
    # Tokenize the input word
    inputs = tokenizer(word, return_tensors='pt')
    
    # Disable gradient calculation
    with torch.no_grad():
        # Pass the tokenized input through the model
        outputs = model(**inputs)
    
    # Extract the embeddings
    embeddings = outputs.last_hidden_state
    
    # Convert token IDs back to tokens
    tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0])
    
    # Store the embeddings in a dictionary
    word_embeddings = {}
    for i, token in enumerate(tokens):
        if token == word:
            word_embeddings[token] = embeddings[0, i, :].numpy()
            break
    
    return word_embeddings



# Find the substitute words for the original keyword, by iterating over the standard word list
def substitute_word(word):
    json_dir = "/Users/lucasma/Documents/The States/ECBC/Code/ECBCData2024/standardizedwords.json"
    with open (json_dir, "r") as f:
        standardWord = json.load(f)
    ret = []
    for term, equals in standardWord.items():
        for spell in equals:
            if spell==word:
                ret.append(term)
    return ret

def clean_embedding(embeddings):
    clean = {}
    for token, embedding in embeddings.items():
        if (token in tokenizer.all_special_tokens) or (token.lower() in stop_words) or (token in string.punctuation) or (token.startswith('##')) or (token=="•"):
            continue
        clean[token] = embedding
    return clean

# Main function
def find_top_similar_words(target_words, sentences, tokenizer, model, top_n):
    
    embeddings = clean_embedding(get_word_embedding(sentences, tokenizer, model))
    key_sim_word = {}

    for keyWord in target_words:
        presence = ""
        wordFound = False

        if keyWord in embeddings:
            presence = keyWord
            wordFound = True
        else:
            for equal in substitute_word(keyWord):
                wordFound = wordFound or equal in embeddings
                if wordFound:
                    presence = equal
                    break
        
        if not wordFound:
            key_embedding =  get_single_embedding(keyWord, tokenizer, model)[keyWord]
        else:
            key_embedding = embeddings[presence]

        similarities = {}

        for token, embedding in embeddings.items():
            similarities[token] = np.dot(embedding, key_embedding) / ((np.linalg.norm(embedding)) * (np.linalg.norm(key_embedding)))
    
        # Use a heap to find the top N similar words
        top_entries = []

        for token, similarity in similarities.items():
                heapq.heappush(top_entries, (similarity, token))

                if len(top_entries) > top_n:
                    removed_sim, removed_t = heapq.heappop(top_entries)

        top_entries.sort(reverse=True, key=lambda x: x[0])

        key_sim_word[keyWord] = top_entries

    return key_sim_word


# Path to the .txt file
file_path = '/Users/lucasma/Documents/The\ States/ECBC/Code/ECBCData2024/data/VirginiaTotal.txt'
# file_path = "data/A10010_cleaned.txt"

sentences = read_sentence_document(file_path)

# Target word to find similarities with
target_words = ["money", "christ", "light", "darkness", "clothes", "naked"]

# Find and print top 10 similar words
top_similar_words = find_top_similar_words(target_words, sentences, tokenizer, model, 50)

for word, words in top_similar_words.items():
    print("######################################################")
    print(f"Top 50 words most similar to '{word}':")
    for (sim, relWord) in words:
        print(f"{relWord}: {sim}")


######################################################
Top 50 words most similar to 'money':
1.0000001192092896: money
0.933540403842926: wealth
0.9297288060188293: silver
0.9293482899665833: folly
0.9277276992797852: debt
0.925916850566864: favour
0.9251276850700378: drunken
0.9249783754348755: punishment
0.9233909845352173: worthy
0.9230401515960693: doctrine
0.9217734336853027: treasure
0.9207563996315002: property
0.9207444190979004: reason
0.920539379119873: plenty
0.9201928377151489: portion
0.9197781682014465: buy
0.9195746779441833: penny
0.9193478226661682: river
0.9188104867935181: fight
0.9187880158424377: wicked
0.9186394810676575: journey
0.9173088669776917: misery
0.9171804189682007: shilling
0.9166380167007446: building
0.9166153073310852: build
0.9160724878311157: drunkenness
0.9160028696060181: water
0.9159512519836426: people
0.9156801104545593: price
0.9154247045516968: happy
0.9152998328208923: duty
0.9148253798484802: ceremony
0.9147508144378662: merchant
0.9146800